# EDA — Day 4 (Plotly)

Sections:
1. Setup + imports
2. Load processed data
3. Quick audit
4. Questions + results
5. Bootstrap comparison
6. Findings + caveats



In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px

# Resolve project root robustly (supports running from repo root or notebooks/)
ROOT = Path().resolve()
if not (ROOT / "data" / "processed").exists():
    ROOT = ROOT.parent
DATA = ROOT / "data/processed/analytics_table.parquet"
FIGS = ROOT / "reports/figures"
FIGS.mkdir(parents=True, exist_ok=True)
print("Using ROOT=", ROOT)


def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires kaleido)."""

    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)



Using ROOT= C:\Users\fayez\bootcamp\week2\w2_DataWork


In [2]:
df = pd.read_parquet(DATA)

print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print("\nTop missing columns:\n", missing)



rows: 5250 cols: 18
order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
year                          float64
month                  string[python]
dow                            object
hour                          float64
country                        object
dtype: object

Top missing columns:
 quantity              534
status                513
status_clean          513
dow                   507
hour                  507
created_at            507
date                  507
year                  507
month                 507
amount__is_outlier    495
dtype: int64


### Questions

1. Revenue by country
2. Revenue over time (by month)
3. Distribution of amount_winsor
4. Refund rate by country


1. ما هو إيرادات وعدد الطلبات ومتوسط قيمة الطلب لكل دولة؟
2. كيف تتغير الإرادات شهرياً عبر السنة؟
3. ما هو شكل توزيع المبالغ بعد الـ winsorizing؟

أسلوب إحصائي  للتعامل مع القيم المتطرفة بدون حذفها.

4. (Bootstrap)  متوسط المبلغ بين طلبات السعودية والإمارات؟

أسلوب قوي لفهم عدم اليقين في النتائج عندما لا نعرف شكل التوزيع الحقيقي للبيانات أو عندما تكون العينة صغيرة.


In [4]:
rev = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean"),
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)

fig = px.bar(rev, x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
save_fig(fig, FIGS / "revenue_by_country.png")
rev.head()



,country,n,revenue,aov
0,AE,1348,324843.68,257.607994
1,KW,1326,307761.92,260.153779
2,QA,1344,292329.93,244.014967
3,SA,1232,286976.39,257.840422


**Interpretation — Revenue by country**

- AE tops revenue (~325k) with AOV ~258; KW and QA follow, SA close behind.
- Volumes are similar across countries (n ~1.2–1.35k), so revenue ranking is not driven by outlier counts.
- Caveat: amounts include winsorized values; check extreme orders separately if doing finance-grade reporting.



In [5]:
trend = (
    df.groupby("month", dropna=False)
    .agg(n=("order_id", "size"), revenue=("amount", "sum"))
    .reset_index()
    .sort_values("month")
)

fig = px.line(trend, x="month", y="revenue", title="Revenue over time (monthly)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
save_fig(fig, FIGS / "revenue_trend_monthly.png")
trend.head()



,month,n,revenue
0,2025-01,431,106530.04
1,2025-02,381,86507.35
2,2025-03,394,91068.07
3,2025-04,437,101450.03
4,2025-05,424,99063.3


**Interpretation — Monthly revenue trend**

- Monthly revenue ranges ~64k–118k; earliest months (Jan ~106k) lead, while Dec dips (~64k).
- A small <NA> bucket (~118k) comes from rows with missing/invalid month; worth cleaning if time analyses matter.
- Caveat: created_at had missing values before parsing; timing gaps could bias month-level comparisons.



In [6]:
fig = px.histogram(df, x="amount_winsor", nbins=30, title="Order amount distribution (winsorized)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
save_fig(fig, FIGS / "amount_hist_winsor.png")
fig



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'bingroup': 'x',
              'hovertemplate': 'amount_winsor=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'nbinsx': 30,
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': {'bdata': ('j8L1KFz3fUApXI/C9ShfQAAAAAAAAP' ... 'zMzMxceECPwvUoXI9jQDMzMzMzs0dA'),
                    'dtype': 'f8'},
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Order amount distribution (winsorized)', 'x': 0.02},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Amount (winsorized)'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of orders'}}}
})

**Interpretation — Amount distribution**

- Winsorized amounts center around median ≈257 with IQR ≈130–377 and long right tail capped at ~495.
- Typical orders cluster between ~130 and ~380; extreme orders are limited by winsorization to reduce skew.
- Caveat: Winsorization hides true high-end outliers; for fraud/finance reviews use uncapped amounts too.



In [8]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())

    diffs = np.array(diffs)
    ci_low, ci_high = np.quantile(diffs, [0.025, 0.975])
    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(ci_low),
        "ci_high": float(ci_high),
        "n_a": int(len(a)),
        "n_b": int(len(b)),
    }



In [9]:
group_a = df.loc[df["country"] == "SA", "amount"]
group_b = df.loc[df["country"] == "AE", "amount"]
res = bootstrap_diff_means(group_a, group_b, n_boot=2000, seed=42)
print(res)



{'diff_mean': 0.2324286262916644, 'ci_low': -11.07714188528196, 'ci_high': 11.66253862078395, 'n_a': 1113, 'n_b': 1261}


In [10]:
avg_sa = df[df["country"] == "SA"]["amount"].mean()
avg_ae = df[df["country"] == "AE"]["amount"].mean()

print(f"🇸🇦 متوسط مبيعات السعودية: {avg_sa:.2f}")
print(f"🇦🇪 متوسط مبيعات الإمارات: {avg_ae:.2f}")


🇸🇦 متوسط مبيعات السعودية: 257.84
🇦🇪 متوسط مبيعات الإمارات: 257.61


**Bootstrap comparison — SA vs AE (amount)**

- Diff_mean (SA−AE) ≈ +0.23; 95% CI ≈ [−11.08, +11.66], so the difference is small/uncertain.
- Sample sizes: n_SA=1113, n_AE=1261; groups are well-populated for bootstrapping.
- Caveat: amounts are winsorized; uncapped amounts could widen the CI slightly.



### Findings (draft)

- AE تقود الإيراد (~325k) يليها KW وQA؛ الفروق ليست بسبب حجم الطلبات فقط لأن n متقارب بين الدول.
- الإيراد الشهري يتراوح تقريباً 64k–118k مع انخفاض في ديسمبر؛ هناك حزمة <NA> (~118k) نتيجة تواريخ مفقودة.
- المبالغ winsorized تتمحور حول ميديان ~257 وبمدى نموذجي ~130–380 مع ذيل أيمن محدود إلى ~495.
- Bootstrap (SA−AE) لمتوسط المبلغ ≈ +0.23 مع CI [−11.08, +11.66] → الفرق غير حاسم إحصائياً مع عينات كافية.
- Caveat: بعض `created_at` مفقود، والمبالغ مقصوصة (winsorized) مما قد يخفي قيم قصوى مؤثرة في التحليل المالي.

